In [2]:
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np

In [4]:
tracks_df = pd.read_csv('cleaned_tracks_dataset.csv')
user_tracks_df = pd.read_csv('users_tracks_cleaned.csv')

In [5]:
unique_tracks = tracks_df['track_id'].unique()
user_track_matrix = pd.DataFrame(0, index=user_tracks_df['user_id'], columns=unique_tracks)

for index, row in user_tracks_df.iterrows():
    user_id = row['user_id']
    for track_id in row[1:]:
        user_track_matrix.loc[user_id, track_id] = 1

nmf_model = NMF(n_components=10, init='nndsvd', random_state=42, max_iter=500)
user_factors = nmf_model.fit_transform(user_track_matrix)
track_factors = nmf_model.components_

In [6]:
predictions = {}

for user_id in user_track_matrix.index:
    if user_id < len(user_factors):
        user_preferences = np.dot(user_factors[user_id - 1], track_factors)

        user_fav_tracks = user_tracks_df[user_tracks_df['user_id'] == user_id].values[0][1:]
        recommendations = [
            (track, score)
            for track, score in zip(user_track_matrix.columns, user_preferences)
            if track not in user_fav_tracks
        ]

        top_recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:3]
        predictions[user_id] = [track for track, score in top_recommendations]

predictions_df = pd.DataFrame.from_dict(predictions, orient='index', columns=['rec_1', 'rec_2', 'rec_3'])

In [7]:

predictions_df.to_csv('user_recommendations_NMF.csv', index_label='user')